In [0]:
from pyspark.sql import SparkSession 

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import sha2, concat_ws, upper, col, expr
from pyspark.sql.types import StructType, StructField, LongType, StringType, DateType, IntegerType

In [0]:
spark = SparkSession.builder.appName('FactSales2Dimensionality').getOrCreate()

In [0]:
df = spark.read.format("delta").load("/delta/fact_sales_final")

In [0]:
display(df)

ProductCategory,ProductName,Brand,StoreRegion,StoreName,StoreType,SalesRep,Department,EmployeeRole,UnitsSold,UnitPrice,Discount,SaleDate
Furniture,T-shirt,BrandB,East,StoreX,Franchise,Martha Long,Electronics,Cashier,12.0,279.35,5.0,2022-12-14
Clothing,Tablet,BrandC,East,StoreZ,Franchise,Martha Long,Home,Sales Associate,33.67,272.49,0.0,2023-02-24
Clothing,Tablet,BrandA,South,StoreX,Retail,Emily Vazquez,Apparel,Cashier,33.67,484.75,15.0,2025-03-24
Electronics,Smartphone,BrandB,West,StoreY,Outlet,Charles Fields,Apparel,Cashier,26.0,205.74,10.0,2023-09-30
Furniture,T-shirt,BrandC,East,StoreZ,Outlet,Wendy Castillo,Home,Manager,46.0,20.25,5.0,2022-10-14
Furniture,T-shirt,BrandC,South,StoreY,Retail,Wendy Castillo,Home,Manager,22.5,361.06,10.0,2024-02-23
Clothing,T-shirt,BrandC,South,StoreY,Outlet,John Harris,Home,Cashier,37.0,492.65,5.0,2024-05-06
Electronics,Smartphone,BrandC,South,StoreX,Outlet,Charles Fields,Home,Sales Associate,37.0,293.87,15.0,2023-04-04
Clothing,Jeans,BrandA,South,StoreY,Retail,Wendy Castillo,Electronics,Manager,23.0,189.47,15.0,2022-12-26
Furniture,T-shirt,BrandB,East,StoreZ,Franchise,Charles Fields,Apparel,Manager,25.0,359.08,10.0,2022-10-28


In [0]:
df1 = spark.read.format("delta").load("/delta/dim_product")


In [0]:
display(df1)

ProductCategory,ProductName,Brand,DIM_ProductId
Electronics,Smartphone,BrandC,fc08e15f7ca6c6df9df7da9e4815aaf89a29e1e19459ee7d969019e8cbb37123
Electronics,Smartphone,BrandB,6a421110b266104bf1cf67be336c9ca4cfff1f897b231e0631948bb3f034677e
Electronics,T-shirt,BrandA,cd2f205a3b37acc265cf4867714abf35098ab215ba53c4864cd498abeda41f19
Electronics,Desk,BrandC,d77d44bf969beb3dbe1c220e980137a5a50d692fcb5488788dcab5776eea5935
Clothing,Tablet,BrandA,a8406b374712399cebdf16a3e8a8c56cf72bf72db75cb2ad0dc3a1ff71e61285
Electronics,Chair,BrandA,dcd1602d12efc29c8c4598b7843ec57a534683119f15c2042695e5fe71031ffd
Furniture,Desk,BrandA,b5c80882f8e29da43355e05bb4fef15d73e22fc3e8a99209a389132cb1eaa6f2
Furniture,Tablet,BrandA,1839298fc3616a7fb8581b8eb5373248e28632ab00dfdec326053bc309f58bca
Electronics,Desk,BrandA,27280aade539db2d7433159d56c762fc2f0d86f6c030d9edc55eaaf355997161
Electronics,Tablet,BrandB,36fc80bfe93a526d8a175456e8243770785451a22d4ce473e3bbf391080eeba1


In [0]:
df2 = spark.read.format("delta").load("/delta/dim_store_final")

In [0]:
display(df2)

StoreRegion,StoreName,StoreType,DIM_StoreId
North,StoreY,Outlet,94ba623007bdf38a8d9dd25b359ba7a49b0a434a8c49e2aaff76cc12c50a8a77
South,StoreY,Retail,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072
East,StoreX,Retail,eaaa9336e570e385b77b0050a85962732193ca23004af89ae72f2889abd91bd3
North,StoreX,Outlet,cd1aac1b6cedfa5d51c325a3c19d2132ce133ac285ce754fdf436527f68ab882
North,StoreZ,Retail,30d46cf1deb74c2d33b4ea7e7f6798ae0623d5195c080721dbf022942ba48b72
South,StoreZ,Franchise,b697ea824f0541eb515498c5eb6f2ce43947880d645dbe09ee14075c732aef48
South,StoreZ,Retail,1120c453edb7620e31fdc4578bc15c8cb5aa664c7d30218eefcaec0b67e511ce
East,StoreX,Franchise,3c7aa2361bb113e0892437ecba18d0fca00119aefb95552a6b18bb7eee14759a
South,StoreX,Outlet,76868e571d2e34422cf3c39e724b18da8fe589d29e3bd589ee5390c548aaabd5
East,StoreZ,Franchise,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df


In [0]:
df3 = spark.read.format("delta").load("/delta/dim_employee_final")

In [0]:
display(df3)

SalesRep,Department,EmployeeRole,DIM_EmployeeId
Kyle Lin,Electronics,Sales Associate,b90eb8c52995b65d912ee69813964cf8b27d10fea22e5176a0efde3a2c331ad0
Charles Fields,Apparel,Manager,11e3df9450f3951d15425406cfd773e170e7e53297b071b8dfcfa87a688599c3
Wendy Castillo,Home,Manager,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab
Wendy Castillo,Electronics,Manager,5b35a50e37f2fb896eef5da16148195eff3e208575ad22d8a6edfbff8511201c
Charles Fields,Home,Cashier,8b9d3284385f39a45911d6eb2bb4611a58d6ec152bee163ad1e886bd1083cd46
Kyle Lin,Home,Cashier,e3729a173a8c99d2fbdf176c56d695c1576bcc8e1a39a695a626c2fb1badb7a2
John Harris,Electronics,Manager,87ac6d5a4977bf75fdd34375ba02238b096ce6503cb4b6c9cd785114665a4d03
John Harris,Apparel,Manager,880162d4763b81e4ac57db4b957d788ae22f0ff2f3673c6d7c15f3a56a63ea54
Billy Perez,Electronics,Cashier,0b98f44b61cd1b506c8069d037184c4fde9d6b5b111aaa06ecb519d90d626be7
John Harris,Home,Cashier,fdedff2cb72a6a9be5669b3ceb5a76b5954802411dda3d1e83ddb183d5a86548


In [0]:
#Start with base fact table
df_fact = df

#1. Join with Product Dimension to get DIM_ProductId
df_fact = df_fact.join(
    df1.select("ProductCategory", "ProductName", "Brand", "DIM_ProductId"),
    on=["ProductCategory", "ProductName", "Brand"],
    how="left"
)

#2. Join with Store Dimension to get DIM_StoreId
df_fact = df_fact.join(
    df2.select("StoreRegion", "StoreName", "StoreType", "DIM_StoreId"),
    on=["StoreRegion", "StoreName", "StoreType"],
    how="left"
)

#3. Join with Employee Dimension to get DIM_EmployeeId
df_fact = df_fact.join(
    df3.select("SalesRep", "Department", "EmployeeRole", "DIM_EmployeeId"),
    on=["SalesRep", "Department", "EmployeeRole"],
    how="left"
)

In [0]:
display(df_fact)

SalesRep,Department,EmployeeRole,StoreRegion,StoreName,StoreType,ProductCategory,ProductName,Brand,UnitsSold,UnitPrice,Discount,SaleDate,DIM_ProductId,DIM_StoreId,DIM_EmployeeId
Martha Long,Electronics,Cashier,East,StoreX,Franchise,Furniture,T-shirt,BrandB,12.0,279.35,5.0,2022-12-14,0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,3c7aa2361bb113e0892437ecba18d0fca00119aefb95552a6b18bb7eee14759a,7e6b7470e0c5045a1797405ca77f04a0ba3c1e2e2e88bf125d9a33d10b597eac
Martha Long,Home,Sales Associate,East,StoreZ,Franchise,Clothing,Tablet,BrandC,33.67,272.49,0.0,2023-02-24,c37d4ae373d30210c6e3b8760d2e14d3f1ce92979c432aa3ccef3493b5ef97b8,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,0f0a2f01281855e40cd8674f9192e6736b34aa8fc5f0478d71361f8a8412ba56
Emily Vazquez,Apparel,Cashier,South,StoreX,Retail,Clothing,Tablet,BrandA,33.67,484.75,15.0,2025-03-24,a8406b374712399cebdf16a3e8a8c56cf72bf72db75cb2ad0dc3a1ff71e61285,d99aca1849c5560475ec973a6220e019f1b32f02cf85be1fdad144ec31466970,fcabe27a77b7b0062b0c99723dbfa1e531629cccb118cb71f9c41d22a9aba099
Charles Fields,Apparel,Cashier,West,StoreY,Outlet,Electronics,Smartphone,BrandB,26.0,205.74,10.0,2023-09-30,6a421110b266104bf1cf67be336c9ca4cfff1f897b231e0631948bb3f034677e,84b0247bf1c9e21f1a7befa453f14c708b5be10d1f352da6b89183f8d691ecc2,c93c0a4829fd929a76a46896a25c78a5e14ba88c21b0e526dbc45c4e5378ea90
Wendy Castillo,Home,Manager,East,StoreZ,Outlet,Furniture,T-shirt,BrandC,46.0,20.25,5.0,2022-10-14,8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,4bdb0c80a77b7ad25194de65cb257a855aef4714db3c3562712c670228caa9da,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab
Wendy Castillo,Home,Manager,South,StoreY,Retail,Furniture,T-shirt,BrandC,22.5,361.06,10.0,2024-02-23,8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab
John Harris,Home,Cashier,South,StoreY,Outlet,Clothing,T-shirt,BrandC,37.0,492.65,5.0,2024-05-06,86ebe6b6839b1f0b802dc9acb8b3ac28ed84edd9496e774602b4bb858ac1ee21,411dea8e689cd912cd11682ee1f8b3ddbe7a4b413e89be652757c39f04f2ec60,fdedff2cb72a6a9be5669b3ceb5a76b5954802411dda3d1e83ddb183d5a86548
Charles Fields,Home,Sales Associate,South,StoreX,Outlet,Electronics,Smartphone,BrandC,37.0,293.87,15.0,2023-04-04,fc08e15f7ca6c6df9df7da9e4815aaf89a29e1e19459ee7d969019e8cbb37123,76868e571d2e34422cf3c39e724b18da8fe589d29e3bd589ee5390c548aaabd5,27d0986b4172fcb4be66444087e6bced12fe736c8e1a53d6f5144aea5778742c
Wendy Castillo,Electronics,Manager,South,StoreY,Retail,Clothing,Jeans,BrandA,23.0,189.47,15.0,2022-12-26,807d270019a32fa7c40a2b8931312f136363e6bf9793576c7ab67fd901472887,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,5b35a50e37f2fb896eef5da16148195eff3e208575ad22d8a6edfbff8511201c
Charles Fields,Apparel,Manager,East,StoreZ,Franchise,Furniture,T-shirt,BrandB,25.0,359.08,10.0,2022-10-28,0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,11e3df9450f3951d15425406cfd773e170e7e53297b071b8dfcfa87a688599c3


In [0]:
df_fact.schema

Out[30]: StructType([StructField('SalesRep', StringType(), True), StructField('Department', StringType(), True), StructField('EmployeeRole', StringType(), True), StructField('StoreRegion', StringType(), True), StructField('StoreName', StringType(), True), StructField('StoreType', StringType(), True), StructField('ProductCategory', StringType(), True), StructField('ProductName', StringType(), True), StructField('Brand', StringType(), True), StructField('UnitsSold', DoubleType(), True), StructField('UnitPrice', DoubleType(), True), StructField('Discount', DoubleType(), True), StructField('SaleDate', DateType(), True), StructField('DIM_ProductId', StringType(), True), StructField('DIM_StoreId', StringType(), True), StructField('DIM_EmployeeId', StringType(), True)])

In [0]:
df_fact.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/delta/df_fact")

In [0]:
df_fact_cleaned = df_fact.select(
    "DIM_ProductId",
    "DIM_StoreId",
    "DIM_EmployeeId",
    "UnitsSold",
    "UnitPrice",
    "Discount",
    "SaleDate"
)

In [0]:
display(df_fact_cleaned)

DIM_ProductId,DIM_StoreId,DIM_EmployeeId,UnitsSold,UnitPrice,Discount,SaleDate
0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,3c7aa2361bb113e0892437ecba18d0fca00119aefb95552a6b18bb7eee14759a,7e6b7470e0c5045a1797405ca77f04a0ba3c1e2e2e88bf125d9a33d10b597eac,12.0,279.35,5.0,2022-12-14
c37d4ae373d30210c6e3b8760d2e14d3f1ce92979c432aa3ccef3493b5ef97b8,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,0f0a2f01281855e40cd8674f9192e6736b34aa8fc5f0478d71361f8a8412ba56,33.67,272.49,0.0,2023-02-24
a8406b374712399cebdf16a3e8a8c56cf72bf72db75cb2ad0dc3a1ff71e61285,d99aca1849c5560475ec973a6220e019f1b32f02cf85be1fdad144ec31466970,fcabe27a77b7b0062b0c99723dbfa1e531629cccb118cb71f9c41d22a9aba099,33.67,484.75,15.0,2025-03-24
6a421110b266104bf1cf67be336c9ca4cfff1f897b231e0631948bb3f034677e,84b0247bf1c9e21f1a7befa453f14c708b5be10d1f352da6b89183f8d691ecc2,c93c0a4829fd929a76a46896a25c78a5e14ba88c21b0e526dbc45c4e5378ea90,26.0,205.74,10.0,2023-09-30
8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,4bdb0c80a77b7ad25194de65cb257a855aef4714db3c3562712c670228caa9da,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab,46.0,20.25,5.0,2022-10-14
8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab,22.5,361.06,10.0,2024-02-23
86ebe6b6839b1f0b802dc9acb8b3ac28ed84edd9496e774602b4bb858ac1ee21,411dea8e689cd912cd11682ee1f8b3ddbe7a4b413e89be652757c39f04f2ec60,fdedff2cb72a6a9be5669b3ceb5a76b5954802411dda3d1e83ddb183d5a86548,37.0,492.65,5.0,2024-05-06
fc08e15f7ca6c6df9df7da9e4815aaf89a29e1e19459ee7d969019e8cbb37123,76868e571d2e34422cf3c39e724b18da8fe589d29e3bd589ee5390c548aaabd5,27d0986b4172fcb4be66444087e6bced12fe736c8e1a53d6f5144aea5778742c,37.0,293.87,15.0,2023-04-04
807d270019a32fa7c40a2b8931312f136363e6bf9793576c7ab67fd901472887,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,5b35a50e37f2fb896eef5da16148195eff3e208575ad22d8a6edfbff8511201c,23.0,189.47,15.0,2022-12-26
0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,11e3df9450f3951d15425406cfd773e170e7e53297b071b8dfcfa87a688599c3,25.0,359.08,10.0,2022-10-28


In [0]:
df_fact_cleaned.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/delta/df_fact_cleaned")


In [0]:
df_fact_cleaned = df_fact_cleaned.withColumn("NetRevenue",(col("UnitsSold") * col("UnitPrice") * (1 - col("Discount"))))

In [0]:
display(df_fact_cleaned)

DIM_ProductId,DIM_StoreId,DIM_EmployeeId,UnitsSold,UnitPrice,Discount,SaleDate,NetRevenue
0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,3c7aa2361bb113e0892437ecba18d0fca00119aefb95552a6b18bb7eee14759a,7e6b7470e0c5045a1797405ca77f04a0ba3c1e2e2e88bf125d9a33d10b597eac,12.0,279.35,5.0,2022-12-14,-13408.800000000001
c37d4ae373d30210c6e3b8760d2e14d3f1ce92979c432aa3ccef3493b5ef97b8,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,0f0a2f01281855e40cd8674f9192e6736b34aa8fc5f0478d71361f8a8412ba56,33.67,272.49,0.0,2023-02-24,9174.7383
a8406b374712399cebdf16a3e8a8c56cf72bf72db75cb2ad0dc3a1ff71e61285,d99aca1849c5560475ec973a6220e019f1b32f02cf85be1fdad144ec31466970,fcabe27a77b7b0062b0c99723dbfa1e531629cccb118cb71f9c41d22a9aba099,33.67,484.75,15.0,2025-03-24,-228501.45500000002
6a421110b266104bf1cf67be336c9ca4cfff1f897b231e0631948bb3f034677e,84b0247bf1c9e21f1a7befa453f14c708b5be10d1f352da6b89183f8d691ecc2,c93c0a4829fd929a76a46896a25c78a5e14ba88c21b0e526dbc45c4e5378ea90,26.0,205.74,10.0,2023-09-30,-48143.159999999996
8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,4bdb0c80a77b7ad25194de65cb257a855aef4714db3c3562712c670228caa9da,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab,46.0,20.25,5.0,2022-10-14,-3726.0
8ff5e4961ecd6be9af8b4301fd055a9fb74cc9d9345e154247fdb1ba69cc53aa,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,c4a16c19bcbaa9d8754690e3a800e18c1e1ee9c48dde85e5b64bcf7f30578cab,22.5,361.06,10.0,2024-02-23,-73114.65000000001
86ebe6b6839b1f0b802dc9acb8b3ac28ed84edd9496e774602b4bb858ac1ee21,411dea8e689cd912cd11682ee1f8b3ddbe7a4b413e89be652757c39f04f2ec60,fdedff2cb72a6a9be5669b3ceb5a76b5954802411dda3d1e83ddb183d5a86548,37.0,492.65,5.0,2024-05-06,-72912.2
fc08e15f7ca6c6df9df7da9e4815aaf89a29e1e19459ee7d969019e8cbb37123,76868e571d2e34422cf3c39e724b18da8fe589d29e3bd589ee5390c548aaabd5,27d0986b4172fcb4be66444087e6bced12fe736c8e1a53d6f5144aea5778742c,37.0,293.87,15.0,2023-04-04,-152224.66
807d270019a32fa7c40a2b8931312f136363e6bf9793576c7ab67fd901472887,24007c12903a7fd1b36a5faa4c4069a2be77b043fb7b335f4a558f650aa68072,5b35a50e37f2fb896eef5da16148195eff3e208575ad22d8a6edfbff8511201c,23.0,189.47,15.0,2022-12-26,-61009.340000000004
0ee892f6bcd5fe280d3d4ccfd72a138675d0514a58e1335c6a48b02823bb8156,20f1ebafb8d3513848d887c5554eb0ce0ea2cc393afc88041813c6f9714ee7df,11e3df9450f3951d15425406cfd773e170e7e53297b071b8dfcfa87a688599c3,25.0,359.08,10.0,2022-10-28,-80793.0


In [0]:
df_fact_cleaned.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/delta/df_fact_cleaned")


In [0]:
invalid_products = df_fact_cleaned.join(
    df1.select("DIM_ProductId"),
    on="DIM_ProductId",
    how="left_anti"
)

print("Invalid DIM_ProductId rows in FACT:")
invalid_products.show()

Invalid DIM_ProductId rows in FACT:
+-------------+-----------+--------------+---------+---------+--------+--------+----------+
|DIM_ProductId|DIM_StoreId|DIM_EmployeeId|UnitsSold|UnitPrice|Discount|SaleDate|NetRevenue|
+-------------+-----------+--------------+---------+---------+--------+--------+----------+
+-------------+-----------+--------------+---------+---------+--------+--------+----------+



In [0]:
invalid_stores = df_fact_cleaned.join(
    df2.select("DIM_StoreId"),
    on="DIM_StoreId",
    how="left_anti"
)

print("Invalid DIM_StoreId rows in FACT:")
invalid_stores.show()

Invalid DIM_StoreId rows in FACT:
+-----------+-------------+--------------+---------+---------+--------+--------+----------+
|DIM_StoreId|DIM_ProductId|DIM_EmployeeId|UnitsSold|UnitPrice|Discount|SaleDate|NetRevenue|
+-----------+-------------+--------------+---------+---------+--------+--------+----------+
+-----------+-------------+--------------+---------+---------+--------+--------+----------+



In [0]:
invalid_employees = df_fact_cleaned.join(
    df3.select("DIM_EmployeeId"),
    on="DIM_EmployeeId",
    how="left_anti"
)

print("Invalid DIM_EmployeeId rows in FACT:")
invalid_employees.show()

Invalid DIM_EmployeeId rows in FACT:
+--------------+-------------+-----------+---------+---------+--------+--------+----------+
|DIM_EmployeeId|DIM_ProductId|DIM_StoreId|UnitsSold|UnitPrice|Discount|SaleDate|NetRevenue|
+--------------+-------------+-----------+---------+---------+--------+--------+----------+
+--------------+-------------+-----------+---------+---------+--------+--------+----------+



Every table showed empty columns and only header which shows all surrogate keys in df_fact_cleaned match correctly with their respective dimension tables.
This is because :
||left_anti join is empty	 Passed	 Every DIM_*Id in FACT exists in the respective DIM table||
||No missing keys	Passed	successfully replaced natural keys with surrogate keys||
||Data integrity preserved	Confirmed	Foreign key relationships are correctly established||
